In [ ]:
#!pip install matplotlib
#!pip install torch
#!pip install gym

In [ ]:
#Import neccessary libraries
import gym
from gym import spaces
import numpy as np
import copy
import numpy as np
import math as m
import random as r
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
"""
from IPython import display
from collections import namedtuple
import torch.nn as nn
import torch.optim as optim
from collections import deque
from itertools import count
import torch.nn.functional as F
"""

In [ ]:
import gym
# Register the environment
gym.register(
    id='vehicleRouting-v0',
    entry_point='Julian_1cyc_Zelda_my_vre:vehicleRoutingEnv',
    kwargs={'current_customer_list': None, 'current_truck_capacity': None, 'total_truck_capacity': None, 'depot_location': None}
)

In [ ]:
def normalized_instance_creator(cust_max_dem, truck_cap, num_cust, num_row, num_col):
    depot = (r.random(),r.random())
    occupied_coords=[]
    occupied_coords.append(depot)
    current_customer_list=[]
    current_customer_list.append(depot)
    for cust in range(num_cust):
        customer=[]
        coord=depot
        while coord in occupied_coords:
            coord=(r.random(),r.random())
        occupied_coords.append(coord)
        customer.append(coord)
        demand=(r.randint(1,cust_max_dem))
        customer.append(demand)
        distance_to_depot=dist = (m.sqrt(((depot[0]-coord[0])**2)+(depot[1]-coord[1])**2))
        customer.append(distance_to_depot)
        distance_to_truck=distance_to_depot
        customer.append(distance_to_truck)
        position_to_depot=(coord[0]-depot[0],coord[1]-depot[1])
        customer.append(position_to_depot)
        position_to_truck=position_to_depot
        customer.append(position_to_truck)
        current_customer_list.append(customer)
    current_truck_capacity=truck_cap
    total_truck_capacity=truck_cap
    return current_customer_list, current_truck_capacity, total_truck_capacity, depot

In [ ]:
#Generate random problem instance, generating random values (some within range, like demand) for matrices and locations
#self, info_vector, customer_location_matrix, truck_location_matrix, customer_demand_matrix, truck_capacity_matrix, customer_timeframe_matrix, truck_timeframe_matrix, customer_location_tuple_list
#Generate instance function would be entirely seperate from solver
#Outer creates problem instance, then calls solve entire problem, input tuple of location
def instance_creator(cust_max_dem, truck_cap, num_cust, num_row, num_col):
    depot = (r.randint(0,num_row-1),r.randint(0,num_col)-1)
    occupied_coords=[]
    occupied_coords.append(depot)
    current_customer_list=[]
    current_customer_list.append(depot)
    for cust in range(num_cust):
        customer=[]
        coord=depot
        while coord in occupied_coords:
            coord=(r.randint(0,num_row-1),r.randint(0,num_col)-1)
        occupied_coords.append(coord)
        customer.append(coord)
        demand=(r.randint(1,cust_max_dem))
        customer.append(demand)
        distance_to_depot=dist = (m.sqrt(((depot[0]-coord[0])**2)+(depot[1]-coord[1])**2))
        customer.append(distance_to_depot)
        distance_to_truck=distance_to_depot
        customer.append(distance_to_truck)
        position_to_depot=(coord[0]-depot[0],coord[1]-depot[1])
        customer.append(position_to_depot)
        position_to_truck=position_to_depot
        customer.append(position_to_truck)
        current_customer_list.append(customer)
    current_truck_capacity=truck_cap
    total_truck_capacity=truck_cap
    return current_customer_list, current_truck_capacity, total_truck_capacity, depot
    

## Define the problem values

In [ ]:
#Defining problem instance variables
#Changing the numbers, changes the single instance NN is training

# KEVIN
cust_max_dem = 5
truck_cap = 10
num_cust = 2
num_row = 10
num_col = 10

## Create instance and initialize environment

In [ ]:
#Creating problem instance using defined variables
current_customer_list, current_truck_capacity, total_truck_capacity, depot_location = \
instance_creator(cust_max_dem, truck_cap, num_cust, num_row, num_col)

In [ ]:
print(f"Current Customer List: {current_customer_list}")
print(f"Depot: {depot_location}")
print(current_truck_capacity)

In [ ]:
#KEVIN
current_customer_list=[(5, 1),
 [(9, 3), 5, 4.47213595499958, 4.47213595499958, (4, 2), (4, 2)],
 [(8, 1), 3, 3.0, 3.0, (3, 0), (3, 0)]]
depot_location = (5,1)
current_truck_capacity=10
total_truck_capacity=10

In [ ]:
# Test the environment
# Making the enviromenet, using the created matrices from the creeated problem instance
env = gym.make('vehicleRouting-v0', current_customer_list=current_customer_list, \
               current_truck_capacity=current_truck_capacity, \
               total_truck_capacity=total_truck_capacity, depot_location=depot_location)
#Checks if you are in ipython enviroment
is_ipython = 'inline' in matplotlib.get_backend()
#If true, then import display which is used later for the visual plots
if is_ipython:
    from IPython import display
#Makes the plots interactive, so that the plots are visualized and changing as new data is being created
plt.ion()
#Check if there is dedicated GPU, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
#Defining the environment action space number and reset environment
n_actions=env.action_space.n
state, info =env.reset()

In [ ]:
#Testing the render function
env.render()

In [ ]:
#Defining transition
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

#Creating replay memory class
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))
#        new_transition = Transition(*args)
#        # Check for an existing transition with matching state and action
#        match_found = False
#        for existing_transition in self.memory:
#            state_match = torch.equal(existing_transition.state, new_transition.state)
#            action_match = torch.equal(existing_transition.action, new_transition.action)
#            if state_match and action_match:
#                match_found = True
#                break
#        if not match_found:
#            self.memory.append(new_transition)

    def sample(self, batch_size):
        return r.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
#Defining DQN class, input layer is the observations (flattened states list)
#Two layers of 128 nodes each
#Output layer of the number of actions
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        print(f'*** Initializing with {n_observations}, {n_actions}')
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 32)
        self.ln1 = nn.LayerNorm(32)
        self.layer2 = nn.Linear(32, 16)
        self.ln2 = nn.LayerNorm(16)
        self.layer3 = nn.Linear(16, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.ln1(self.layer1(x)))
        x = F.relu(self.ln2(self.layer2(x)))
#        x = F.relu(self.layer3(x))
        return self.layer3(x)

## Test Single Step

In [ ]:
def select_action(state, exclude_indices):
    global steps_done
    global global_eps_threshold

    DEBUG_SELECT= False
    sample = r.random()
#Eps threshold is used for exploitation vs exploration
#When the sample is greater than the calculated eps threshold, then the NN is taking the policy selected action (exploitation)
#When the sample is not greater, then the NN is choosing a random action (exploration)
#The ending eps value is added to the difference between the starting and ending epsilon value, which would
#This is multiplied by e^(-1 times the number of steps over the decay value of 1000). Meaning that as the number
#of steps increase than -1 is multiplied by a larger fraction and therefore, the expression, m.exp(-1. * steps_done / EPS_DECAY)
#is becoming a smaller fraction. And since the fraction is being multplied by eps end and the difference of end and start,
#the entire express is decreasing over time, and therefore, more samples are greater than the threshold leading
#to more exploitation
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        m.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    global_eps_threshold = eps_threshold
    # DEBUG
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            # Create a mask for the actions to exclude
            action_values = policy_net(state)
            #print(f"action_values: {action_values}")
            #Create array of boolean values for the mask
            exclude_mask = torch.full_like(action_values, float('0'), dtype=torch.bool)
            #Sets corresponding values in row 0 of exclude mask to True if exclude indicies indicates it
            exclude_mask[0,exclude_indices] = True
            #print(f"exclude_mask: {exclude_mask}")
            #Sets masked out corresponding values to negative infinity so that no matter what, the network will not choose them
            #because of their high negative reward (negative inf)
            masked_action_values = torch.where(exclude_mask, torch.tensor(float('-inf')), action_values)
            if DEBUG_SELECT:
                print(f"Masked Action Values: {masked_action_values}")
            return masked_action_values.max(1).indices.view(1,1)
            # return policy_net(state).max(1).indices.view(1, 1)
    else:
        sample_action = env.action_space.sample()
        while sample_action in exclude_indices:
            sample_action = env.action_space.sample()
        return torch.tensor([[sample_action]], device=device, dtype=torch.long)


episode_costs = []
episode_losses= []

def plot_costs(title_str = '',show_result=False):
    plt.figure(1)
    costs_t = torch.tensor(episode_costs, dtype=torch.float)
    losses = torch.tensor(episode_losses, dtype=torch.float)
    epoches= list(range(1,len(costs_t)+1))

    if show_result == False:
        plt.clf()
    fig, ax1 = plt.subplots()
    
    # Plot the first series on ax1
    color = 'tab:blue'
    if show_result:
        ax1.set_title(f'Result ({title_str})')
    else:
        ax1.set_title(f'Training... ({title_str})')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Negative Cost', color=color)
    if len(costs_t) >=100:
        ax1.plot(epoches[101:], costs_t[101:], color=color)
        ax1.tick_params(axis='y', labelcolor=color)
        means = costs_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        ax1.plot(epoches[101:], means[101:].numpy(), color='tab:orange')
    else:
        ax1.plot(epoches, costs_t, color=color)
        ax1.tick_params(axis='y', labelcolor=color)
    
    
    # Create ax2 that shares the same x-axis with ax1
    ax2 = ax1.twinx()  
    color = 'tab:green'
    ax2.set_ylabel('Nnet Loss', color=color)  
    if len(costs_t) >=100:
        means = losses.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        ax2.plot(epoches[101:], means[101:].numpy(), color=color)
        ax2.tick_params(axis='y', labelcolor=color)
    else:
        ax2.plot(epoches, losses, color=color)
        ax2.tick_params(axis='y', labelcolor=color)
    
    fig.tight_layout()  # To ensure a neat layout
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

def orig_plot_costs(title_str = '',show_result=False):
    plt.figure(1)
    costs_t = torch.tensor(episode_costs, dtype=torch.float)
    losses = torch.tensor(episode_losses, dtype=torch.float)

    if show_result:
        plt.title(f'Result ({title_str})')
    else:
        plt.clf()
        plt.title(f'Training... ({title_str})')
    plt.xlabel('Episode')
    plt.ylabel('Cost')
    if len(costs_t) >= 100:
        plt.plot(costs_t[101:].numpy())
        plt.plot(losses[101:].numpy())
    else:
        plt.plot(costs_t.numpy())
        plt.plot(losses.numpy())
        
    # Take 100 episode averages and plot them too
    if len(costs_t) >= 100:
        means = costs_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means[101:].numpy())
#    plt.pause(0.0001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [ ]:
def opt():
    DEBUG_OPT=True
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    # with torch.no_grad():
    #    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values

    with torch.no_grad():
        action_values = target_net(non_final_next_states)
        exclude_mask = torch.full_like(action_values, float('0'), dtype=torch.bool)
        #Sets corresponding values in row 0 of exclude mask to True if exclude indicies indicates it
#        exclude_indices = [env.get_invalid_ones_w_states(e.tolist()) for e in non_final_next_states.to('cpu')]
        exclude_indices=[]
        for row_idx, cols in enumerate(exclude_indices):
            for col_idx in cols:
                exclude_mask[row_idx, col_idx] = True
        masked_action_values = torch.where(exclude_mask, torch.tensor(float('-inf')), action_values)
        next_state_values[non_final_mask] = masked_action_values.max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    if DEBUG_OPT and opt_cnt%100:
        print(f"state: {state_batch}")
        pritn(f"action: {action_batch}")
        print(f"reward: {reward_batch}")
        print(f"expected_state_action_values: {expected_state_action_values}")
        print(f"state_action_values: {state_action_values}")
        pritn(f"loss: {loss}")

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
def optimize_model():
    global opt_cnt;
    global opt_loss;
    DEBUG = False
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    # with torch.no_grad():
    #    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values

    with torch.no_grad():
        action_values = target_net(non_final_next_states)
        exclude_mask = torch.full_like(action_values, float('0'), dtype=torch.bool)
        #Sets corresponding values in row 0 of exclude mask to True if exclude indicies indicates it
        exclude_indices = [env.get_invalid_ones_w_states(e.tolist()) for e in non_final_next_states.to('cpu')]
        for row_idx, cols in enumerate(exclude_indices):
            for col_idx in cols:
                exclude_mask[row_idx, col_idx] = True
        masked_action_values = torch.where(exclude_mask, torch.tensor(float('-inf')), action_values)
        next_state_values[non_final_mask] = masked_action_values.max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    opt_loss = loss
    opt_cnt+=1
    if DEBUG and opt_cnt%100 == 1:
        print(f"opt_cnt: {opt_cnt} ******************************************************************************************************************************************")
        print(f"state: {state_batch}")
        print(f"action: {action_batch.tolist()}")
        print(f"reward: {reward_batch}")
        print(f"masked_action_values: {masked_action_values}")
        print(f"next_state_values: {next_state_values}")
        print(f"expected_state_action_values: {expected_state_action_values}")
        print(f"state_action_values: {state_action_values.tolist()}")
        print(f"loss: {loss}")
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

## Run x number of episodes and plot

In [ ]:
#Reset NN
#Initialize parameters
#BATCH_SIZE = 128
BATCH_SIZE = 64 # was 256
GAMMA = 0.99
EPS_START = 0.9 # was 0.9
EPS_END = 0.0 # was 0.05
EPS_DECAY = 1000 # was 1000
TAU = 0.02 # was 0.05
LR = 1e-3 # MODIFIED from 1e-5
#LR = 1e-6 (worked)
REPLAY_MEM_SIZE = 1000 # was 1000
# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5)
memory = ReplayMemory(REPLAY_MEM_SIZE)


steps_done = 0
opt_cnt = 0
opt_loss=0.0

In [ ]:
current_customer_list

In [ ]:
#Train the neural network
#If dedicated GPU is found, run 6000 episodes, else run 600 episodes
if torch.cuda.is_available():
    num_episodes = 2000
else:
    num_episodes = 800
best_cost = -1e16
best_mean_cost=-1e16
best_episode = 0
best_mean_episode = 0
last_best_episode = 0
best_path = []
DEBUG = False
PLOT = True
long_path=[]
long_path_visual=[]
long_visited=[]

long_cost = 0
flat_states=[]
global_eps_threshold = 1.0
longest_path=[]
longest_path_cnt=0

for i_episode in range(num_episodes):
    """
    if i_episode%5==0:
    #Creating problem instance using defined variables
        info_vector, customer_location_matrix, truck_location_matrix, \
        customer_demand_matrix, truck_capacity_matrix, customer_location_list = \
        instance_creator(cust_max_dem, truck_cap, num_cust, num_row, num_col)
        #For this instance, set starting truck location to depot location
        truck_location_matrix[info_vector[0][0],info_vector[0][1]]=1
        truck_start_location = (np.where(truck_location_matrix==1)[0][0], np.where(truck_location_matrix==1)[1][0])
        #Set at the location of the truck, the starting demand to the max truck capacity
        truck_capacity_matrix[info_vector[0][0],info_vector[0][1]]=info_vector[1]
        state, info = env.update(info_vector, customer_location_matrix, truck_location_matrix, customer_demand_matrix,\
        truck_capacity_matrix, customer_location_list, truck_start_location)
    # Initialize the environment and get it's state
    else:
    """
    state, info = env.reset()
    env.reorder_customers()
    env.update()
    state = env.get_state()
    if DEBUG: 
        print(f"[{i_episode}] State: {state} *************************************************************\n\n")
        print(f"Longest path ({longest_path_cnt}) {longest_path}")
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    
    total_cost = 0.0
    visited = [0]
    path=[]
    rewards=[]
    current_lr = optimizer.param_groups[0]['lr']
    for t in count():
        #Print information for debugging and training
        invalid_ones = env.get_invalid_ones()
        if DEBUG:
            print(f"{t} -------------------------------------------------")
            print(f'invalid_ones = {invalid_ones}')
        action = select_action(state, invalid_ones)
        if DEBUG: 
            print(f"action selected: {action.item()}")
            print(f"path so far: {path}") 
        observation, reward, terminated, truncated, info = env.step(action.item())
        if reward <= -env.max_dist:
            reward = -1.0
        if DEBUG: 
            print(f"Step done ---------------------")
            print(f"cur_cust_list: {env.current_customer_list}")
            print(f"observation: {observation}")
            print(f"reward: {reward}")
            print(f"terminated: {terminated}, truncated: {truncated}")
        v = env.get_visited()
        u = env.get_unvisited()
        if DEBUG: 
            print(f"visited: {v}")
            print(f"unvisited: {u}")
        total_cost = total_cost + reward
        visited = copy.deepcopy(info["visited"])
        if action.item() not in visited:
            visited.append(action.item())
        rewards.append(-reward)
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated
        visited = copy.deepcopy(info["visited"])
        unvisited = copy.deepcopy(info["unvisited"])
        if action.item()==2:
            path.append([action.item(), visited, unvisited])
        else:
            path.append([action.item(), "no customer selected", ""])
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
        # Store the transition in memory
        memory.push(state, action, next_state, reward)
        if state is not None:
            temp = [t.tolist() for t in flat_states]
            #print(f"temp: {temp}")
            if state.tolist() not in temp:
                flat_states.append(state)
        if next_state is not None:
            temp = [t.tolist() for t in flat_states]
            if next_state.tolist() not in temp:
                flat_states.append(next_state)
        # Move to the next state
        state = next_state
        #print(f"state: {state}")
        
        #print(f"action: {action.item()}")
        # print(f"Memory length: {len(memory)}")
       # Perform one step of the optimization (on the policy network)
        if DEBUG:
            print("Calling optimize_model ...")
        optimize_model()
        if DEBUG:
            print("Done optimize model.")
       # Soft update of the target network's weights
       # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict: 
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)            
        if done:
            if DEBUG:
                print(f"******** Total cost: {total_cost}")
            episode_costs.append(total_cost)
            episode_losses.append(opt_loss)
            # TODO: the code below has no effect
            memory_size = len(memory)
            if memory_size > REPLAY_MEM_SIZE:
                replay = True
            else:
                replay = False
            mean = np.mean(episode_costs[-100:])
            if len(path) > longest_path_cnt:
                longest_path_cnt=len(path)
                longest_path=path
            if total_cost == best_cost:
                last_best_episode=i_episode
            if total_cost>best_cost:
                best_cost=total_cost
                best_path=path
                best_episode = i_episode
                last_best_episode=i_episode
            if mean>best_mean_cost:
                best_mean_cost=mean
                best_mean_episode=i_episode
            if total_cost<long_cost:
                long_cost= total_cost
                long_path=path
                long_path_visual=[]
                for value in long_path:
                    if value[0]==1:
                        long_path_visual.append("Right")
                        long_path_visual.append(value[1])
                    elif value[0]==2:
                        long_path_visual.append("Select")
                        long_path_visual.append(value[1])
                    elif value[0]==3:
                        long_path_visual.append("Depot")
                        long_path_visual.append(value[1])
                long_visited=visited
                long_rewards=rewards
            title_str = f'[{global_eps_threshold:.4f}: {current_lr:.8f}: {opt_loss:.6f}: {long_cost:.4f}/{best_episode}/{last_best_episode}/{i_episode}] [{best_mean_episode}] Mean Cost:{best_mean_cost:.4f}/{mean:.4f}, Best_Cost: {best_cost:.4f}, Longest_Path_Length: {longest_path_cnt}'
            if DEBUG == False:
                if PLOT == True:
                    plot_costs(title_str=title_str)
            break
    scheduler.step()
print(f"[{best_episode}: Best cost: {best_cost} and best path: {best_path}")
print('Complete')
# HERE
if DEBUG == False:
    if PLOT == True:
        plot_costs(show_result=True) 
        plt.ioff()
        plt.show()

In [ ]:
long_path

In [ ]:
model_filename='layernorm-normalize-debug-2cust-scheduler-nodedop-to_zero_explore.nnet'
torch.save(policy_net.state_dict(), model_filename)

In [ ]:
observation

In [ ]:
long_path_visual

In [ ]:
long_cost

In [ ]:
current_customer_list

In [ ]:
env.current_customer_list

In [ ]:
env.max_dist

In [ ]:
14.94427190999916

In [ ]:
env.reset()

In [ ]:
dist1=4.47213595499958/env.max_dist; dist2=3/env.max_dist; print(f"dist1: {dist1}, dist2: {dist2}")

In [ ]:
long_path

In [ ]:
best_cost

In [ ]:
best_path

In [ ]:
flat_states observation: [0, 0, 1, 5, 1, 9, 3, 0, 4.47213595499958, 2.23606797749979, 4, 2, 1, 2, 8, 1, 0, 3.0, 0.0, 3, 0, 0, 0, 7, 10]


In [ ]:
current_customer_list

In [ ]:
best_path

In [ ]:
long_path_visual

In [ ]:
long_path

In [ ]:
lists = [t.tolist() for t in flat_states]
for i in lists:
    print(i)
    

In [ ]:
policy_net

In [ ]:
long_path

In [ ]:
len(long_path)

In [ ]:
long_visited

In [ ]:
len(long_path)*0.001

In [ ]:
max_length = 0
current_length = 0

index=0
marked=False
lengths=[]
for num in long_path:
    # Check if the current number is 1
    if num == 1 or num==0:
        # Increase the length of the current sequence
        current_length += 1
        # Update the maximum length if the current sequence is longer
        if current_length>max_length:
            max_length=current_length
            if marked == False:
                max_index=index
            else:
                marked=True
    else:
        lengths.append(current_length)
        # Reset the current sequence length if the current number is not 1
        current_length = 0
        marked=False
    index=index+1    
print(max_length)

In [ ]:
lengths

In [ ]:
max_index

In [ ]:
len(long_path)

In [ ]:
long_path[279716]

In [ ]:
max_length

In [ ]:
current_customer_list

In [ ]:
for i in range(1,len(current_customer_list)):
    x=current_customer_list[0][0]
    y=current_customer_list[0][1]
    dist = m.sqrt((x-current_customer_list[i][0][0])**2+(y-current_customer_list[i][0][1])**2)
    print(f"[{i}] {dist}")

In [ ]:
b

In [ ]:
total_cost

## Single Run of the trained model

In [ ]:
if torch.cuda.is_available():
    num_episodes = 40000
else:
    num_episodes = 800
DEBUG = True
PLOT = False
flat_states=[]
global_eps_threshold = 1.0
i_episode = 10000
state, info = env.reset()
env.reorder_customers()
env.update()
state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
total_cost = 0.0
visited = [0]
current_lr = optimizer.param_groups[0]['lr']
path=[]
if DEBUG:
    print(f"Initial state: {state}")
for t in count():
    #Print information for debugging and training
    invalid_ones = env.get_invalid_ones()
    if DEBUG:
        print(f"{t} -------------------------------------------------")
        print(f'invalid_ones = {invalid_ones}')
    action = select_action(state, invalid_ones)
    if DEBUG: 
        print(f"action selected: {action.item()}")
        print(f"path so far: {path}") 
    observation, reward, terminated, truncated, info = env.step(action.item())
    if DEBUG: 
        print(f"Step done ---------------------")
        print(f"cur_cust_list: {env.current_customer_list}")
        print(f"observation: {observation}")
        print(f"reward: {reward}")
        print(f"terminated: {terminated}, truncated: {truncated}")
    v = env.get_visited()
    u = env.get_unvisited()
    if DEBUG: 
        print(f"visited: {v}")
        print(f"unvisited: {u}")
    total_cost = total_cost + reward
    visited = copy.deepcopy(info["visited"])
    if action.item() not in visited:
        visited.append(action.item())
    reward = torch.tensor([reward], device=device)
    done = terminated or truncated
    visited = copy.deepcopy(info["visited"])
    unvisited = copy.deepcopy(info["unvisited"])
    if action.item()==2:
        path.append([action.item(), visited, unvisited])
    else:
        path.append([action.item(), "no customer selected", ""])
    if terminated:
        next_state = None
    else:
        next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
    # Move to the next state
    state = next_state
    if done:
        if DEBUG:
            print(f"******** Total cost: {total_cost}")
        if DEBUG == False:
            if PLOT == True:
                plot_costs(title_str=title_str)
        break
    scheduler.step()
print('Complete')
# HERE
if DEBUG == False:
    if PLOT == True:
        plot_costs(show_result=True) 
        plt.ioff()
        plt.show()

[0, 1, 0, 5, 1, 9, 3, 5, 4.47213595499958, 4.47213595499958, 4, 2, 4, 2, 8, 1, 3, 3.0, 3.0, 3, 0, 3, 0, 10, 10]

In [ ]:
env.max_dist

## Test the trained network

In [ ]:
#Reset environment
state, info = env.reset()
visited = [0]
state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

total_cost = 0  # Variable to store the total reward for this episode
actions_taken = []  # List to store the actions taken

In [ ]:
#See the state info
state

In [ ]:
env.get_invalid_ones()

In [ ]:
#Run first step
action = select_action(state, visited)
print(f"action selected: {action}")
observation, reward, terminated, truncated, info = env.step(action.item())
print(f"observation: {observation}")
print(f"reward: {reward}")
print(f"terminated: {terminated}, truncated: {truncated}")
v = env.get_visited()
u = env.get_unvisited()
print(f"visited: {v}")
print(f"unvisited: {u}")
total_cost = total_cost + reward
visited = copy.deepcopy(info["visited"])
if action.item() not in visited:
    visited.append(action.item())
done = terminated or truncated
print(f"Total cost so far: {total_cost}")

In [ ]:
#Second step
action = select_action(state, visited)
print(f"action selected: {action}")
observation, reward, terminated, truncated, info = env.step(action.item())
print(f"observation: {observation}")
print(f"reward: {reward}")
print(f"terminated: {terminated}, truncated: {truncated}")
v = env.get_visited()
u = env.get_unvisited()
print(f"visited: {v}")
print(f"unvisited: {u}")
total_cost = total_cost + reward
visited = copy.deepcopy(info["visited"])
if action.item() not in visited:
    visited.append(action.item())
done = terminated or truncated
print(f"Total cost so far: {total_cost}")

In [ ]:
#Third step
action = select_action(state, visited)
print(f"action selected: {action}")
observation, reward, terminated, truncated, info = env.step(action.item())
print(f"observation: {observation}")
print(f"reward: {reward}")
print(f"terminated: {terminated}, truncated: {truncated}")
v = env.get_visited()
u = env.get_unvisited()
print(f"visited: {v}")
print(f"unvisited: {u}")
total_cost = total_cost + reward
visited = copy.deepcopy(info["visited"])
if action.item() not in visited:
    visited.append(action.item())
done = terminated or truncated
print(f"Total cost so far: {total_cost}")

In [ ]:
#Fourth step
action = select_action(state, visited)
print(f"action selected: {action}")
observation, reward, terminated, truncated, info = env.step(action.item())
print(f"observation: {observation}")
print(f"reward: {reward}")
print(f"terminated: {terminated}, truncated: {truncated}")
v = env.get_visited()
u = env.get_unvisited()
print(f"visited: {v}")
print(f"unvisited: {u}")
total_cost = total_cost + reward
visited = copy.deepcopy(info["visited"])
if action.item() not in visited:
    visited.append(action.item())
done = terminated or truncated
print(f"Total cost so far: {total_cost}")

In [ ]:
#Fifth step
action = select_action(state, visited)
print(f"action selected: {action}")
observation, reward, terminated, truncated, info = env.step(action.item())
print(f"observation: {observation}")
print(f"reward: {reward}")
print(f"terminated: {terminated}, truncated: {truncated}")
v = env.get_visited()
u = env.get_unvisited()
print(f"visited: {v}")
print(f"unvisited: {u}")
total_cost = total_cost + reward
visited = copy.deepcopy(info["visited"])
if action.item() not in visited:
    visited.append(action.item())
done = terminated or truncated
print(f"Total cost so far: {total_cost}")

In [ ]:
#Sixth step
action = select_action(state, visited)
print(f"action selected: {action}")
observation, reward, terminated, truncated, info = env.step(action.item())
print(f"observation: {observation}")
print(f"reward: {reward}")
print(f"terminated: {terminated}, truncated: {truncated}")
v = env.get_visited()
u = env.get_unvisited()
print(f"visited: {v}")
print(f"unvisited: {u}")
total_cost = total_cost + reward
visited = copy.deepcopy(info["visited"])
if action.item() not in visited:
    visited.append(action.item())
done = terminated or truncated
print(f"Total cost so far: {total_cost}")

In [ ]:
#Seventh step
action = select_action(state, visited)
print(f"action selected: {action}")
observation, reward, terminated, truncated, info = env.step(action.item())
print(f"observation: {observation}")
print(f"reward: {reward}")
print(f"terminated: {terminated}, truncated: {truncated}")
v = env.get_visited()
u = env.get_unvisited()
print(f"visited: {v}")
print(f"unvisited: {u}")
total_cost = total_cost + reward
visited = copy.deepcopy(info["visited"])
if action.item() not in visited:
    visited.append(action.item())
done = terminated or truncated
print(f"Total cost so far: {total_cost}")

In [ ]:
#Eigth step
action = select_action(state, visited)
print(f"action selected: {action}")
observation, reward, terminated, truncated, info = env.step(action.item())
print(f"observation: {observation}")
print(f"reward: {reward}")
print(f"terminated: {terminated}, truncated: {truncated}")
v = env.get_visited()
u = env.get_unvisited()
print(f"visited: {v}")
print(f"unvisited: {u}")
total_cost = total_cost + reward
visited = copy.deepcopy(info["visited"])
if action.item() not in visited:
    visited.append(action.item())
done = terminated or truncated
print(f"Total cost so far: {total_cost}")

In [ ]:
#Ninth step
action = select_action(state, visited)
print(f"action selected: {action}")
observation, reward, terminated, truncated, info = env.step(action.item())
print(f"observation: {observation}")
print(f"reward: {reward}")
print(f"terminated: {terminated}, truncated: {truncated}")
v = env.get_visited()
u = env.get_unvisited()
print(f"visited: {v}")
print(f"unvisited: {u}")
total_cost = total_cost + reward
visited = copy.deepcopy(info["visited"])
if action.item() not in visited:
    visited.append(action.item())
done = terminated or truncated
print(f"Total cost so far: {total_cost}")